In [16]:
import cv2
import math
import numpy as np

In [17]:
# 映像のキャプチャ
cap = cv2.VideoCapture("shot0_1001.mp4")

In [18]:
# 映像プロパティの取得
# float で取得される
print("幅 : ", cap.get(cv2.CAP_PROP_FRAME_WIDTH))
print("高さ : ", cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print("フレームレート : ", cap.get(cv2.CAP_PROP_FPS))
print("フレーム数 : ", cap.get(cv2.CAP_PROP_FRAME_COUNT))

幅 :  768.0
高さ :  576.0
フレームレート :  25.0
フレーム数 :  179.0


In [19]:
# ビデオクリップ数
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
if frame_count >= 64:
    frames_per_clip = 64
elif frame_count < 64 and frame_count >= 32:
    frames_per_clip = 32
else:
    print("This viodeo is too short...")
clip_num = math.ceil(frame_count / frames_per_clip)
duplicate_frams = [0] + [(frames_per_clip * clip_num - frame_count + i) // (clip_num - 1) for i in range(clip_num - 1)]

In [20]:
duplicate_frams

[0, 6, 7]

In [21]:
# フレーム画像の切り出し
frame_image_list = []
while True:
    ret, frame = cap.read()
    if ret:
        frame_image_list.append(frame)
    else:
        break

In [22]:
len(frame_image_list)

179

In [23]:
# ビデオクリップのフレームの番号を選択
frame_idx_list = []
for clip_idx in range(clip_num):
    first_frame_idx = clip_idx * frames_per_clip - sum(duplicate_frams[:clip_idx + 1])
    if frames_per_clip == 64:
        frame_idx = [first_frame_idx + i * 2 for i in range(32)]
    else:
        frame_idx = [first_frame_idx + i for i in range(32)]
    frame_idx_list.append(frame_idx)

In [25]:
# ビデオクリップを作成する
video_clip_list = []
for frame_indexes in frame_idx_list:
    frame_images = [cv2.cvtColor(frame_image_list[idx], cv2.COLOR_BGR2RGB) for idx in frame_indexes]
    video_clip_list.append(frame_images)

In [26]:
print(len(video_clip_list))
print(len(video_clip_list[0]))
print(video_clip_list[0][0].shape)

3
32
(576, 768, 3)


In [27]:
def split_video(video_path):
    # 映像のプロパティを取得
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # 1ビデオクリップのフレーム数を決める
    if frame_count >= 64:
        frames_per_clip = 64
    elif frame_count < 64 and frame_count >= 32:
        frames_per_clip = 32
    else:
        print("This viodeo is too short...")
        return None
    clip_num = math.ceil(frame_count / frames_per_clip)
    duplicate_frams = [0] + [(frames_per_clip * clip_num - frame_count + i) // (clip_num - 1) for i in range(clip_num - 1)]
    
    # フレーム画像の取得
    frame_image_list = []
    while True:
        ret, frame = cap.read()
        if ret:
            frame_image_list.append(frame)
        else:
            break
    
    # ビデオクリップのフレームの番号を選択
    frame_idx_list = []
    for clip_idx in range(clip_num):
        first_frame_idx = clip_idx * frames_per_clip - sum(duplicate_frams[:clip_idx + 1])
        if frames_per_clip == 64:
            frame_idx = [first_frame_idx + i * 2 for i in range(32)]
        else:
            frame_idx = [first_frame_idx + i for i in range(32)]
        frame_idx_list.append(frame_idx)
        
    # ビデオクリップを作成する
    video_clip_list = []
    for frame_indexes in frame_idx_list:
        frame_images = [cv2.cvtColor(frame_image_list[idx], cv2.COLOR_BGR2RGB) for idx in frame_indexes]
        video_clip_list.append(frame_images)
        
    return video_clip_list

In [29]:
video_clip_list = split_video("shot0_1001.mp4")

In [32]:
print(len(video_clip_list[0]))

32
